# Epoching for Continuous Data 
Created: 03/14/2024 by A Fink \
Updated: 03/18/2024

In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy


import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/alexandrafink/Documents/GitHub/LFPAnalysis/')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

## Epoch continuous resting state data
- load bipolar rereferenced lfp data
- crop data to 7 minutes, starting from 10 seconds into recording 
- epoch data into 10 second increments (gives 42 epochs)

In [5]:
# Specify root directory for un-archived data and results 
base_dir      = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/resting_state_proj/resting_state_ieeg/'
anat_dir      = f'{base_dir}anat/'
neural_dir    = f'{base_dir}/preprocess/clean_data/'
subj_info_dir = f'{base_dir}patient_tracker/'

subj_ids = list(pd.read_excel(f'{subj_info_dir}subjects_master_list.xlsx', usecols=[0]).PatientID)


In [11]:
bp_lfp_all_subj = {}
epochs_all_subj = {}


for subj_id in subj_ids:
    bp_data = mne.io.read_raw_fif(f'{neural_dir}{subj_id}/{subj_id}_bp_ref_ieeg.fif', preload=False)
    bp_data.crop(tmin=10,tmax=430)
    bp_lfp_all_subj[subj_id] = bp_data
    epochs = mne.make_fixed_length_epochs(bp_data, duration=10, preload=False)
    epochs.save(f'{neural_dir}{subj_id}/{subj_id}_bp_epochs.fif', overwrite=True)
    epochs_all_subj[subj_id] = epochs
    


Opening raw data file /Users/alexandrafink/Documents/GraduateSchool/SaezLab/resting_state_proj/resting_state_ieeg//preprocess/clean_data/MS001/MS001_bp_ref_ieeg.fif...
    Range : 0 ... 304687 =      0.000 ...   609.374 secs
Ready.
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Loading data for 42 events and 5000 original time points ...
0 bad epochs dropped
Loading data for 1 events and 5000 original time points ...
Loading data for 42 events and 5000 original time points ...
Opening raw data file /Users/alexandrafink/Documents/GraduateSchool/SaezLab/resting_state_proj/resting_state_ieeg//preprocess/clean_data/MS003/MS003_bp_ref_ieeg.fif...
    Range : 0 ... 309624 =      0.000 ...   619.248 secs
Ready.
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Loading data for 42 events and 5000 original time points ...
0 bad epochs dropped
Loading data for 1 events and 5000 orig

Opening raw data file /Users/alexandrafink/Documents/GraduateSchool/SaezLab/resting_state_proj/resting_state_ieeg//preprocess/clean_data/MS024/MS024_bp_ref_ieeg.fif...
    Range : 0 ... 303249 =      0.000 ...   606.498 secs
Ready.
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Loading data for 42 events and 5000 original time points ...
0 bad epochs dropped
Loading data for 1 events and 5000 original time points ...
Loading data for 42 events and 5000 original time points ...
Opening raw data file /Users/alexandrafink/Documents/GraduateSchool/SaezLab/resting_state_proj/resting_state_ieeg//preprocess/clean_data/MS025/MS025_bp_ref_ieeg.fif...
    Range : 0 ... 303718 =      0.000 ...   607.436 secs
Ready.
Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Loading data for 42 events and 5000 original time points ...
0 bad epochs dropped
Loading data for 1 events and 5000 orig

In [10]:
epochs_all_subj

{'MS001': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~116 kB, data not loaded,
  '1': 42>,
 'MS003': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~142 kB, data not loaded,
  '1': 42>,
 'MS006': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~125 kB, data not loaded,
  '1': 42>,
 'MS007': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~196 kB, data not loaded,
  '1': 42>,
 'MS008': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~141 kB, data not loaded,
  '1': 42>,
 'MS010': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~137 kB, data not loaded,
  '1': 42>,
 'MS012': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~100 kB, data not loaded,
  '1': 42>,
 'MS014': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~73 kB, data not loaded,
  '1': 42>,
 'MS016': <Epochs |  42 events (good & bad), 0 - 9.998 sec, baseline off, ~117 kB, data not loaded,
  '1'

https://mne.tools/dev/generated/mne.make_fixed_length_epochs.html#mne.make_fixed_length_epochs
https://mne.tools/dev/auto_tutorials/epochs/60_make_fixed_length_epochs.html#sphx-glr-auto-tutorials-epochs-60-make-fixed-length-epochs-py